In [19]:
# %load_ext autoreload
# %autoreload 2

import sys
import os
import pandas as pd
from datetime import datetime, timezone, timedelta

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import src.config as config
from src.inference import get_feature_store


In [27]:
# Use fixed datetime since April data is not available
fixed_hour = df["pickup_hour"].dropna().max()
print(f"📅 Using most recent available hour for predictions: {fixed_hour}")


📅 Using most recent available hour for predictions: 2023-12-31 23:00:00+00:00


In [28]:
import hopsworks

# 🔐 Authenticate with Hopsworks
project = hopsworks.login()

# 🌟 Get the feature store
fs = project.get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=config.FEATURE_GROUP_MODEL_PREDICTION_VERSION
)

# 📦 Read and filter data
df = fg.read()
predictions = df[df["pickup_hour"] == fixed_hour]


2025-05-10 07:26:54,834 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 07:26:54,840 INFO: Initializing external client
2025-05-10 07:26:54,841 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 07:26:55,698 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214682
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.88s) 


In [29]:
top_locations = predictions.sort_values("predicted_rides", ascending=False)
top_locations[["pickup_location_id", "pickup_hour", "predicted_rides"]].head(10)


,pickup_location_id,pickup_hour,predicted_rides
5396,6948.10,2023-12-31 23:00:00+00:00,5
14209,6140.05,2023-12-31 23:00:00+00:00,3
2868,5329.03,2023-12-31 23:00:00+00:00,2
